<a href="https://colab.research.google.com/github/Ebed24/Sport-vision-final/blob/main/Sport_Vision_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook destinado a las funciones finales para el p

**SCRAPE TRANSFERMARKET**

In [7]:
# Import de las librerías

import requests
from bs4 import BeautifulSoup
import time
import random


# Base
import pandas as pd
import numpy as np

# Visualización
import plotly.express as ex

# Web Scraping
import requests
from bs4 import BeautifulSoup

# trabajo con variables abocadas al tiempo
import sys
import time
from datetime import datetime
from termcolor import colored

# GC
import gc

# Itertools
import itertools

# Para extraer datos de un gráfico
from itertools import repeat
from pprint import pprint as pp

# Expresiones regulares
import re

# Configuración
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

In [ ]:
def get_team_links(league_url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

    response = requests.get(league_url, headers=headers)
    if response.status_code != 200:
        print(f"Error al acceder a la página: {league_url}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    teams = []

    # Encontrar la tabla con los equipos
    table = soup.find('table', {'class': 'items'})
    if not table:
        print("No se encontró la tabla de equipos")
        return []

    for row in table.find_all('tr', {'class': ['odd', 'even']}):
        team_cell = row.find('td', {'class': 'hauptlink'})
        if team_cell:
            link = team_cell.find('a', href=True)
            if link:
                team_name = link.get_text(strip=True)
                team_url = 'https://www.transfermarkt.es' + link['href']

                # Eliminar las siglas específicas al principio del nombre del equipo y cualquier información entre paréntesis al final
                team_name = re.sub(r'^(?:CA|CLUB|AA|CSD|CD|CS)\s+|\s*\([^)]*\)$', '', team_name)
                teams.append((team_name, team_url))

    return teams



In [ ]:
# URL de ejemplo (se debe reemplazar por la URL deseada)
league_url = 'https://www.transfermarkt.es/liga-profesional-de-futbol/startseite/wettbewerb/AR1N'
teams = get_team_links(league_url)

# Verificar si se encontraron equipos
print("Equipos encontrados:", teams)

Equipos encontrados: [('River Plate', 'https://www.transfermarkt.es/club-atletico-river-plate/startseite/verein/209/saison_id/2023'), ('Boca Juniors', 'https://www.transfermarkt.es/club-atletico-boca-juniors/startseite/verein/189/saison_id/2023'), ('Racing Club', 'https://www.transfermarkt.es/racing-club/startseite/verein/1444/saison_id/2023'), ('San Lorenzo de Almagro', 'https://www.transfermarkt.es/club-atletico-san-lorenzo-de-almagro/startseite/verein/1775/saison_id/2023'), ('Talleres', 'https://www.transfermarkt.es/club-atletico-talleres/startseite/verein/3938/saison_id/2023'), ('Rosario Central', 'https://www.transfermarkt.es/club-atletico-rosario-central/startseite/verein/1418/saison_id/2023'), ('Vélez Sarsfield', 'https://www.transfermarkt.es/club-atletico-velez-sarsfield/startseite/verein/1029/saison_id/2023'), ('Club Estudiantes de La Plata', 'https://www.transfermarkt.es/club-estudiantes-de-la-plata/startseite/verein/288/saison_id/2023'), ('Argentinos Juniors', 'https://www.t

In [ ]:
def parse_edad(edad_text):
    edad_match = re.search(r'\((\d+)\)', edad_text)
    if edad_match:
        return int(edad_match.group(1))
    else:
        return None

def get_player_data(team_name, team_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(team_url, headers=headers)
    if response.status_code != 200:
        print(f"Error al acceder a la página: {team_url}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'items'})
    if not table:
        print(f"No se encontró la tabla de jugadores para el equipo {team_name}")
        return []

    player_data = {
        'Jugador': [],
        'Posición': [],
        'Edad': [],
        'Nacionalidad': [],
        'Valor de mercado': [],
        'Club': [],
        'Player URL': []
    }

    for row in table.find_all('tr', {'class': ['odd', 'even']}):
        try:
            jugador_cell = row.find('td', {'class': 'hauptlink'})
            if not jugador_cell:
                continue

            jugador_link = jugador_cell.find('a', href=True)
            if not jugador_link:
                continue

            jugador = jugador_link.get_text(strip=True)
            jugador_partial_url = jugador_link['href']

            # Acceder a la página del jugador para obtener la URL completa
            jugador_url = 'https://www.transfermarkt.es' + jugador_partial_url

            pos_table = row.find('table', {'class': 'inline-table'})
            if not pos_table:
                continue

            posicion = pos_table.find_all('tr')[1].find('td').get_text(strip=True)

            edad_cells = row.find_all('td', {'class': 'zentriert'})
            if len(edad_cells) < 3:
                continue

            edad = edad_cells[1].get_text(strip=True)

            nacionalidad_cell = row.find('img', {'class': 'flaggenrahmen'})
            if not nacionalidad_cell:
                continue

            nacionalidad = nacionalidad_cell.get('alt')

            valor_mercado_cell = row.find('td', {'class': 'rechts hauptlink'})
            if not valor_mercado_cell:
                continue

            valor_mercado = valor_mercado_cell.get_text(strip=True)

            player_data['Jugador'].append(jugador)
            player_data['Posición'].append(posicion)
            player_data['Edad'].append(parse_edad(edad))
            player_data['Nacionalidad'].append(nacionalidad)
            player_data['Valor de mercado'].append(valor_mercado)
            player_data['Club'].append(team_name)
            player_data['Player URL'].append(jugador_url)

        except Exception as e:
            print(f"Error procesando fila: {e}")

    return player_data

def convertir_valor(valor):
    if isinstance(valor, str):
        if 'mill. €' in valor:
            return float(valor.replace(' mill. €', '').replace(',', '.')) * 1000000
        elif 'mil €' in valor:
            return float(valor.replace(' mil €', '').replace(',', '.')) * 1000
        else:
            return float(valor.replace(',', '.'))
    else:
        # Si el valor ya es numérico, simplemente devuélvelo
        return valor

In [ ]:
def scrape_transfermarkt(league_url):
    teams = get_team_links(league_url)
    if not teams:
        print("No se encontraron equipos para la liga.")
        return None

    df_list = []
    for team_name, team_url in teams:
        player_data = get_player_data(team_name, team_url)
        if player_data['Jugador']:
            df_list.append(pd.DataFrame(player_data))

    if not df_list:
        print("No se encontraron datos de jugadores.")
        return None

    df = pd.concat(df_list, ignore_index=True)

    df['Valor de mercado'].replace('-', np.nan, inplace=True)
    df['Valor de mercado'].fillna(0, inplace=True)
    df['Valor de mercado'] = df['Valor de mercado'].apply(convertir_valor)

    return df



In [ ]:
# Ejemplo de liga:
league_url = 'https://www.transfermarkt.es/superliga/startseite/wettbewerb/AR1N'
liga_argentina_transfer_urljugador = scrape_transfermarkt(league_url)
if liga_argentina_transfer_urljugador is not None:
    print(liga_argentina_transfer_urljugador)
else:
    print("No se pudieron obtener los datos.")

                Jugador          Posición  Edad Nacionalidad  \
0         Franco Armani           Portero    37    Argentina   
1    Ezequiel Centurión           Portero    27    Argentina   
2       Lucas Lavagnino           Portero    19    Argentina   
3            Paulo Díaz   Defensa central    29        Chile   
4     Sebastián Boselli   Defensa central    20      Uruguay   
..                  ...               ...   ...          ...   
818    Jonathan Herrera  Delantero centro    32    Argentina   
819        Milton Céliz  Delantero centro    31    Argentina   
820        Walter Acuña  Delantero centro    32    Argentina   
821   Gustavo Fernández  Delantero centro    33    Argentina   
822   Maximiliano Brito  Delantero centro    32      Uruguay   

     Valor de mercado         Club  \
0           1000000.0  River Plate   
1            500000.0  River Plate   
2            150000.0  River Plate   
3           5500000.0  River Plate   
4           3500000.0  River Plate   
.. 

In [ ]:
# Guardar el DataFrame en Google Drive
excel_path = '/content/drive/MyDrive/VISION SPORTS/liga_argentina_transfer_urljugador.xlsx'  #copiar la ruta especifca para guardar el documento

liga_argentina_transfer_urljugador.to_excel(excel_path, index=False)


print(f"Datos guardados en {excel_path}")

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/VISION SPORTS'



---


# Scraping de BeSccer

---



In [5]:
url_liga_argentina = 'https://es.besoccer.com/competicion/clasificacion/copa_liga_profesional_argentina/2024/grupo1'

In [3]:
# Hace la busqueda de los equipos de primera division argentina

def scrape_url_liga_argentina(url):
    # Realizamos la petición a la web
    req = requests.get(url)
    time.sleep(1)  # Hacemos una pausa de 1 segundo entre las solicitudes

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:
        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text, "html.parser")

        # Obtenemos todas las tablas con la clase 'table'
        tablas = html.find_all('table', {'class': 'table'})

        # Creamos una lista para almacenar los enlaces
        enlaces_lista = []

        # Recorremos todas las tablas para extraer los enlaces que contienen
        for i, tabla in enumerate(tablas):
            # Buscamos todos los enlaces dentro de la tabla
            enlaces = tabla.find_all('a')
            for j, enlace in enumerate(enlaces):
                # Añadimos el enlace a la lista
                enlaces_lista.append(enlace.get('href'))

        # Filtramos las URLs que contienen '/equipo/' y no contienen '/clasificacion/'
        enlaces_lista = [url for url in enlaces_lista if '/equipo/' in url and '/clasificacion/' not in url]

        # Eliminamos los duplicados convirtiendo la lista en un conjunto (set) y luego de nuevo en una lista
        enlaces_lista = list(set(enlaces_lista))

        # Devolvemos la lista de enlaces
        return enlaces_lista
    else:
        print("Status Code %d" % statusCode)

In [6]:
# Scraping de la primera division argentina / Ejecutamos la función

enlaces_equipos_D1 =scrape_url_liga_argentina(url_liga_argentina)

In [7]:
# Funciona con ambas varibales, agregamos la dirección para que nos envie a la platilla del los equipos

def agregar_plantilla(enlaces):
    # Creamos una nueva lista para almacenar los enlaces modificados
    enlaces_modificados = []

    # Recorremos la lista de enlaces
    for enlace in enlaces:
        # Dividimos el enlace en partes usando '/equipo/' como separador
        partes = enlace.split('/equipo/')

        # Agregamos '/plantilla/' entre '/equipo/' y el nombre del equipo
        enlace_modificado = partes[0] + '/equipo/plantilla/' + partes[1]

        # Añadimos el enlace modificado a la nueva lista
        enlaces_modificados.append(enlace_modificado)

    # Devolvemos la lista de enlaces modificados
    return enlaces_modificados

# Prueba la función con la lista de enlaces
enlaces_modificados = agregar_plantilla(enlaces_equipos_D1)

In [8]:
# Hace la busqueda en cada enlace para obtener todas la url de los jugadores

import concurrent.futures

def scrape_url(url):
    # Usamos una sesión de Requests
    with requests.Session() as s:
        req = s.get(url)
        statusCode = req.status_code
        if statusCode == 200:
            html = BeautifulSoup(req.text, "html.parser")
            entradas = html.find_all('td', {'class': 'name'})
            enlaces = []
            for entrada in entradas:
                enlaces.extend([enlace.get('href') for enlace in entrada.find_all('a')])
            return enlaces
        else:
            print("Status Code %d" % statusCode)
            return []

def scrape_urls(urls):
    todos_enlaces = []
    # Hacemos las solicitudes en paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_url = {executor.submit(scrape_url, url): url for url in urls}
        for future in concurrent.futures.as_completed(future_to_url):
            enlaces = future.result()
            todos_enlaces.extend(enlaces)
    return todos_enlaces


In [9]:
todos_enlaces_jugadores = scrape_urls(enlaces_modificados)


In [10]:
import concurrent.futures

def scrape_url(url):
    # Lista de User-Agents
    user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64)', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)', 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0)']

    # Escogemos un User-Agent al azar
    headers = {'User-Agent': random.choice(user_agents)}

    # Realizamos la petición a la web
    req = requests.get(url, headers=headers)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:
        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text, "html.parser")

        # Obtenemos todos los divs con la clase 'stat'
        divs = html.find_all('div', {'class': 'stat'})

        # Creamos un diccionario para almacenar los resultados
        resultados = {}

        # Recorremos todos los divs para extraer el texto que contienen
        for i, div in enumerate(divs):
            # Extraemos el texto del div
            texto = div.text.strip()

            # Guardamos el texto en el diccionario con la clave 'Div i'
            resultados[f'Div {i + 1}'] = texto

        # Buscamos el nombre del jugador
        jugador = html.find('div', {'class': 'panel-title'})
        if jugador:
            # Extraemos el texto y lo añadimos al diccionario
            resultados['Jugador'] = jugador.text.strip()

    # Buscamos el elemento con la clase 'panel-head ta-c jc-ce'
        panel_head = html.find('div', {'class': 'panel-head ta-c jc-ce'})

        if panel_head:
        # Dentro de ese elemento, buscamos el elemento con la clase 'panel-subtitle'
          panel_subtitle = panel_head.find('div', {'class': 'panel-subtitle'})

        if panel_subtitle:
            # Extraemos el texto y lo añadimos al diccionario
          resultados['Panel Subtitle'] = panel_subtitle.text.strip()

        # Buscamos elementos con la clase 'mb5'
        elementos_mb5 = html.find_all(class_='team-text ta-r')
        if elementos_mb5:
            # Extraemos el texto y lo añadimos al diccionario
            resultados['mb5'] = [elemento.text.strip() for elemento in elementos_mb5]

        return resultados
    else:
        print("Status Code %d" % statusCode)
        return None

def scrape_urls_4(urls):
    todos_resultados = []

    # Hacemos las solicitudes en paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_url = {executor.submit(scrape_url, url): url for url in urls}
        for future in concurrent.futures.as_completed(future_to_url):
            resultados = future.result()
            if resultados is not None:
                todos_resultados.append(resultados)

    # Devolvemos la lista de todos los resultados
    return todos_resultados

In [11]:
def scrape_url(url):
    # Lista de User-Agents
    user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64)', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)', 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0)']

    # Escogemos un User-Agent al azar
    headers = {'User-Agent': random.choice(user_agents)}

    # Realizamos la petición a la web
    req = requests.get(url, headers=headers)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:
        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text, "html.parser")

        # Obtenemos todos los divs con la clase 'stat'
        divs = html.find_all('div', {'class': 'stat'})

        # Creamos un diccionario para almacenar los resultados
        resultados = {}

        # Recorremos todos los divs para extraer el texto que contienen
        for i, div in enumerate(divs):
            # Extraemos el texto del div
            texto = div.text.strip()

            # Guardamos el texto en el diccionario con la clave 'Div i'
            resultados[f'Div {i + 1}'] = texto

        # Buscamos el nombre del jugador
        jugador = html.find('div', {'class': 'panel-title'})
        if jugador:
            # Extraemos el texto y lo añadimos al diccionario
            resultados['Jugador'] = jugador.text.strip()

        # Buscamos el elemento con la clase 'panel-head ta-c jc-ce'
        panel_head = html.find('div', {'class': 'panel-head ta-c jc-ce'})

        if panel_head:
            # Dentro de ese elemento, buscamos el elemento con la clase 'panel-subtitle'
            panel_subtitle = panel_head.find('div', {'class': 'panel-subtitle'})

            if panel_subtitle:
                # Extraemos el texto y lo añadimos al diccionario
                resultados['Panel Subtitle'] = panel_subtitle.text.strip()

        # Buscamos elementos con la clase 'mb5'
        elementos_mb5 = html.find_all(class_='team-text ta-r')
        if elementos_mb5:
            # Extraemos el texto y lo añadimos al diccionario
            resultados['mb5'] = [elemento.text.strip() for elemento in elementos_mb5]

        # Añadimos la URL al diccionario de resultados
        resultados['URL'] = url

        return resultados
    else:
        print("Status Code %d" % statusCode)
        return None


In [12]:
import json
def scrape_ld_json(urls):
    # Definir los encabezados de la solicitud
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Lista para almacenar los resultados
    all_results = []

    for url in urls:
        # Hacer una solicitud GET a la URL
        response = requests.get(url, headers=headers)

        # Crear un objeto BeautifulSoup con el contenido de la respuesta
        soup = BeautifulSoup(response.content, 'html.parser')

        # Buscar todos los bloques de script de tipo "application/ld+json"
        scripts = soup.find_all('script', type='application/ld+json')

        # Extraer y devolver los datos JSON de los scripts
        json_data = [json.loads(script.string) for script in scripts]

        # Extraer los valores de la propiedad "url"
        urls = [data.get('url') for data in json_data if data.get('url') is not None]

        # Retrasar la próxima solicitud
        time.sleep(1)

        # Añadir solo la segunda posición a los resultados
        if len(urls) > 1:
            all_results.append(urls[1])

    return all_results


In [13]:
def scrape_all(urls):
    # Obtener la lista de URLs
    urls_to_scrape = scrape_ld_json(urls)

    # Raspar cada URL
    resultados = scrape_urls_4(urls_to_scrape)

    return resultados


In [14]:
df_2 = scrape_urls_4(todos_enlaces_jugadores)

In [15]:
df_2

[{'Div 1': '27\naños\n\n\n\nArgentina',
  'Div 2': '70\nkgs\n\nDef\n\nposición',
  'Div 3': '180\ncms\n\n3\n\ndorsal',
  'Div 4': '3.67\nM.€\n\n76\n\nELO',
  'Jugador': 'L. Carrizo',
  'Panel Subtitle': 'Lucas Carrizo',
  'mb5': ['CA Huracán\nLiga Profesional Argentina'],
  'URL': 'https://es.besoccer.com/jugador/lucas-carrizo-375268'},
 {'Div 1': '25\naños\n\n\n\nArgentina',
  'Div 2': '75\nkgs\n\nDef\n\nposición',
  'Div 3': '180\ncms\n\n4\n\ndorsal',
  'Div 4': '4.16\nM.€\n\n73\n\nELO',
  'Jugador': 'L. Souto',
  'Panel Subtitle': 'Lucas Alessandro Souto',
  'mb5': ['CA Huracán\nCedido por Defensa y Justicia\nLiga Profesional Argentina'],
  'URL': 'https://es.besoccer.com/jugador/l-souto-874628'},
 {'Div 1': '20\naños\n\n\n\nArgentina',
  'Div 2': '82\nkgs\n\nPor\n\nposición',
  'Div 3': '188\ncms\n\n27\n\ndorsal',
  'Div 4': '4\nK.€\n\n34\n\nELO',
  'Jugador': 'Nazareno Durán',
  'Panel Subtitle': 'Nazareno Durán',
  'mb5': ['CA Huracán\nLiga Profesional Argentina'],
  'URL': 'http

In [19]:
def clean_data(dic_list, keys):
    # Creamos una lista para almacenar todos los resultados
    results = []

    # Recorremos la lista de diccionarios
    for dic in dic_list:
        # Creamos una lista para almacenar las líneas de texto de este diccionario
        lines = []

        # Recorremos la lista de claves
        for key in keys:
            # Si la clave está en el diccionario
            if key in dic:
                # Obtenemos el valor de la clave
                value = dic[key]

                # Verificamos si el valor es una lista
                if isinstance(value, list):
                    # Si es una lista, tratamos cada elemento como una línea separada
                    value_lines = [v.strip() for v in value]
                else:
                    # Si no es una lista, dividimos el valor en líneas
                    value_lines = value.split('\n')

                # Filtramos las líneas para eliminar las que están vacías
                value_lines = [line for line in value_lines if line.strip() != '']

                # Añadimos las líneas a la lista de líneas
                lines.extend(value_lines)

        # Añadimos la lista de líneas a los resultados
        results.append(lines)

    # Devolvemos los resultados
    return results

In [20]:
def procesar_datos(df, keys, columnas_a_eliminar):
    results = clean_data(df, keys)
    df = pd.DataFrame(results)
    df = df.rename(columns={
        0: 'Edad',
        2: 'Nacionalidad',
        3: 'Peso',
        5: 'Posición',
        7: 'Altura',
        11: 'Salario',
        13: 'ELO',
        15: 'Nombre',
        16: 'Equipo',
        17: 'Nombre Completo',
        18: 'URL Jugador'
    })
    df = df.drop(columnas_a_eliminar, axis=1)
    df['Equipo'] = df['Equipo'].str.split('\n').str[0]
    return df

In [21]:
keys = ['Div 1', 'Div 2', 'Div 3', 'Div 4', 'Jugador', 'mb5', 'Panel Subtitle', 'URL']
columnas_a_eliminar = [1,4, 6, 8, 9, 10, 12, 14]
df_procesado = procesar_datos(df_2, keys, columnas_a_eliminar)

In [22]:
df_procesado

,Edad,Nacionalidad,Peso,Posición,Altura,Salario,ELO,Nombre,Equipo,Nombre Completo,URL Jugador
0,27,Argentina,70,Def,180,3.67,76,L. Carrizo,CA Huracán,Lucas Carrizo,https://es.besoccer.com/jugador/lucas-carrizo-...
1,25,Argentina,75,Def,180,4.16,73,L. Souto,CA Huracán,Lucas Alessandro Souto,https://es.besoccer.com/jugador/l-souto-874628
2,20,Argentina,82,Por,188,4,34,Nazareno Durán,CA Huracán,Nazareno Durán,https://es.besoccer.com/jugador/nazareno-duran...
3,24,Argentina,82,Por,180,3.07,60,S. Meza,CA Huracán,Sebastián Tomás Meza,https://es.besoccer.com/jugador/s-meza-919594
4,34,Argentina,85,Def,186,2.55,76,O. Tobio,CA Huracán,Omar Fernando Tobio,https://es.besoccer.com/jugador/f-tobio-72145
...,...,...,...,...,...,...,...,...,...,...,...
917,30,Uruguay,75,Del,186,5.71,75,A. Balboa,Unión Santa Fe,Adrian Martin Balboa Camacho,https://es.besoccer.com/jugador/a-balboa-243381
918,18,Argentina,72,Del,174,4.74,60,Jerónimo Dómina,Unión Santa Fe,Jerónimo Dómina,https://es.besoccer.com/jugador/jeronimo-domin...
919,19,Argentina,-,Del,185,2,32,F. Pumpido,Unión Santa Fe,Francisco Nery Pumpido,https://es.besoccer.com/jugador/f-pumpido-3381319
920,21,Argentina,-,Del,178,1.65,60,Gonzalo Morales,Unión Santa Fe,Gonzalo Javier Morales,https://es.besoccer.com/jugador/javier-morales...




---


# Scraping de Fbref

---



In [8]:
url = "https://fbref.com/es/comps/21/Estadisticas-de-Primera-Division"

In [9]:
# Extraer los nombres de los equipos de esa liga

def get_team_ids_from_url(url):
    # Hacer una solicitud GET a la URL
    response = requests.get(url)
    time.sleep(1)  # Hacer una pausa de 1 segundo

    # Crear un objeto BeautifulSoup con el contenido de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Buscar la primera tabla en la página
    team_table = soup.find_all('table')[1]

    # Buscar todos los enlaces dentro de la tabla
    team_links = team_table.find_all('a')

    # Extraer los ID de los equipos de los enlaces
    team_ids = [link['href'].split('/')[-1] for link in team_links]

    return team_ids

In [10]:
# Extrae los links de las estadisticas de cada equipo

# Headers para simular un navegador web
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

def get_team_links_from_table(url):
    for i in range(5):  # Intenta la solicitud hasta 5 veces
        try:
            # Hacer una solicitud GET a la URL con headers
            response = requests.get(url, headers=headers)
            # Si la solicitud es exitosa, sale del bucle
            break
        except:
            print("Error al hacer la solicitud, reintentando...")
            time.sleep(2)  # Espera 2 segundos antes de reintentar

    # Crear un objeto BeautifulSoup con el contenido de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Buscar la tabla específica en la página
    team_table = soup.find_all('table')[1]  # Cambia el índice según la tabla que quieras analizar

    # Buscar todos los enlaces dentro de la tabla
    team_links = team_table.find_all('a')

    # Extraer los enlaces de los equipos
    team_links = [link['href'] for link in team_links]

    return team_links


In [11]:
team_links = get_team_links_from_table(url)
team_links

['/es/equipos/ceda2145/Estadisticas-de-Talleres',
 '/es/equipos/1d3d37ae/Estadisticas-de-Huracan',
 '/es/equipos/5adc7e67/Estadisticas-de-Union',
 '/es/equipos/8e20e13d/Estadisticas-de-Racing-Club',
 '/es/equipos/950a95f2/Estadisticas-de-Gimnasia-y-Esgrima-LP',
 '/es/equipos/e2f19203/Estadisticas-de-Instituto',
 '/es/equipos/ef99c78c/Estadisticas-de-River-Plate',
 '/es/equipos/d01a653b/Estadisticas-de-Argentinos-Juniors',
 '/es/equipos/11b6dba8/Estadisticas-de-Lanus',
 '/es/equipos/7765008b/Estadisticas-de-Belgrano',
 '/es/equipos/e9ae80b7/Estadisticas-de-Sarmiento',
 '/es/equipos/795ca75e/Estadisticas-de-Boca-Juniors',
 '/es/equipos/8a9d5afa/Estadisticas-de-Independiente-Rivadavia',
 '/es/equipos/df734df9/Estadisticas-de-Estudiantes-LP',
 '/es/equipos/87a920fa/Estadisticas-de-Rosario-Central',
 '/es/equipos/40bb0ce9/Estadisticas-de-Independiente',
 '/es/equipos/42a1ab8b/Estadisticas-de-Atletico-Tucuman',
 '/es/equipos/9bf4eaf4/Estadisticas-de-Newells-Old-Boys',
 '/es/equipos/b3d222b1/

In [12]:
# Base para mezclar con team_links y crear las urls
base_url = "https://fbref.com"

In [13]:
# Función para crear las urls
def crear_urls(base_url, team_links):
    # Crear una lista vacía para almacenar las URLs completas
    urls_completas = []

    # Iterar sobre las rutas de equipo
    for link in team_links:
        # Crear la URL completa uniendo la URL base y la ruta del equipo
        url_completa = base_url + link

        # Agregar la URL completa a la lista
        urls_completas.append(url_completa)

    return urls_completas

In [14]:
urls_completas = crear_urls(base_url, team_links)
urls_completas

['https://fbref.com/es/equipos/ceda2145/Estadisticas-de-Talleres',
 'https://fbref.com/es/equipos/1d3d37ae/Estadisticas-de-Huracan',
 'https://fbref.com/es/equipos/5adc7e67/Estadisticas-de-Union',
 'https://fbref.com/es/equipos/8e20e13d/Estadisticas-de-Racing-Club',
 'https://fbref.com/es/equipos/950a95f2/Estadisticas-de-Gimnasia-y-Esgrima-LP',
 'https://fbref.com/es/equipos/e2f19203/Estadisticas-de-Instituto',
 'https://fbref.com/es/equipos/ef99c78c/Estadisticas-de-River-Plate',
 'https://fbref.com/es/equipos/d01a653b/Estadisticas-de-Argentinos-Juniors',
 'https://fbref.com/es/equipos/11b6dba8/Estadisticas-de-Lanus',
 'https://fbref.com/es/equipos/7765008b/Estadisticas-de-Belgrano',
 'https://fbref.com/es/equipos/e9ae80b7/Estadisticas-de-Sarmiento',
 'https://fbref.com/es/equipos/795ca75e/Estadisticas-de-Boca-Juniors',
 'https://fbref.com/es/equipos/8a9d5afa/Estadisticas-de-Independiente-Rivadavia',
 'https://fbref.com/es/equipos/df734df9/Estadisticas-de-Estudiantes-LP',
 'https://fbr

In [23]:
parte_1=urls_completas[:5]
parte_1

['https://fbref.com/es/equipos/ceda2145/Estadisticas-de-Talleres',
 'https://fbref.com/es/equipos/1d3d37ae/Estadisticas-de-Huracan',
 'https://fbref.com/es/equipos/5adc7e67/Estadisticas-de-Union',
 'https://fbref.com/es/equipos/8e20e13d/Estadisticas-de-Racing-Club',
 'https://fbref.com/es/equipos/950a95f2/Estadisticas-de-Gimnasia-y-Esgrima-LP']

In [94]:
parte_2=urls_completas[5:10]
parte_2

['https://fbref.com/es/equipos/e2f19203/Estadisticas-de-Instituto',
 'https://fbref.com/es/equipos/ef99c78c/Estadisticas-de-River-Plate',
 'https://fbref.com/es/equipos/d01a653b/Estadisticas-de-Argentinos-Juniors',
 'https://fbref.com/es/equipos/11b6dba8/Estadisticas-de-Lanus',
 'https://fbref.com/es/equipos/7765008b/Estadisticas-de-Belgrano']

In [34]:
parte_3=urls_completas[10:15]
parte_3

['https://fbref.com/es/equipos/e9ae80b7/Estadisticas-de-Sarmiento',
 'https://fbref.com/es/equipos/795ca75e/Estadisticas-de-Boca-Juniors',
 'https://fbref.com/es/equipos/8a9d5afa/Estadisticas-de-Independiente-Rivadavia',
 'https://fbref.com/es/equipos/df734df9/Estadisticas-de-Estudiantes-LP',
 'https://fbref.com/es/equipos/87a920fa/Estadisticas-de-Rosario-Central']

In [38]:
parte_4=urls_completas[15:20]
parte_4

['https://fbref.com/es/equipos/40bb0ce9/Estadisticas-de-Independiente',
 'https://fbref.com/es/equipos/42a1ab8b/Estadisticas-de-Atletico-Tucuman',
 'https://fbref.com/es/equipos/9bf4eaf4/Estadisticas-de-Newells-Old-Boys',
 'https://fbref.com/es/equipos/b3d222b1/Estadisticas-de-Deportivo-Riestra',
 'https://fbref.com/es/equipos/41c139b6/Estadisticas-de-Velez-Sarsfield']

In [37]:
parte_5=urls_completas[20:25]
parte_5

['https://fbref.com/es/equipos/3cbfa767/Estadisticas-de-Platense',
 'https://fbref.com/es/equipos/06c1606c/Estadisticas-de-Banfield',
 'https://fbref.com/es/equipos/1d89634b/Estadisticas-de-Barracas-Central',
 'https://fbref.com/es/equipos/ac9a09b4/Estadisticas-de-Godoy-Cruz',
 'https://fbref.com/es/equipos/a4570206/Estadisticas-de-Defensa-y-Justicia']

In [40]:
parte_6=urls_completas[25:]
parte_6

['https://fbref.com/es/equipos/0e92bf17/Estadisticas-de-Tigre',
 'https://fbref.com/es/equipos/66da6009/Estadisticas-de-San-Lorenzo',
 'https://fbref.com/es/equipos/9aa97c75/Estadisticas-de-Central-Cordoba-SdE']

In [16]:
# Extracción de los datos de todos los equipos de la liga Argentina.
import concurrent.futures
def extraccion_automatica(ligas):
    dataframes = []  # Lista para almacenar los DataFrames

    # Crear una sesión de requests
    with requests.Session() as session:
        session.headers = headers

        # Usar un ThreadPoolExecutor para hacer las solicitudes en paralelo
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            future_to_url = {executor.submit(extract_data, url): url for url in ligas}
            for future in concurrent.futures.as_completed(future_to_url):
                df = future.result()
                if df is not None:
                    dataframes.append(df)
                time.sleep(random.randint(2, 5))  # Espera un tiempo aleatorio entre 2 y 5 segundos

    # Concatenar todos los DataFrames a la vez
    df_total = pd.concat(dataframes)

    return df_total


In [17]:
def extract_data(url):
    try:
        # Extraer el nombre del equipo de la URL
        nombre_equipo = url.split('/')[-1]

        # Realizamos la petición a la web
        req = requests.get(url)

        # Comprobamos que la petición nos devuelve un Status Code = 200
        statusCode = req.status_code
        if statusCode == 200:
            # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
            html = BeautifulSoup(req.text, "html.parser")

            # Obtenemos todos los elementos 'tr' dentro de 'tbody'
            tr_elements = html.find('tbody').find_all('tr')

            # Creamos una lista para almacenar los datos de los jugadores
            jugadores = []

            # Recorremos todos los elementos 'tr'
            for tr in tr_elements:
                # Buscamos el elemento 'a' dentro del 'tr'
                a_element = tr.find('a')
                # Si el elemento 'a' existe y tiene un atributo 'href', lo añadimos a los datos del jugador
                if a_element and 'href' in a_element.attrs:
                    jugador = {}
                    jugador['Nombre'] = a_element.text
                    jugador['Enlace'] = a_element['href']
                    jugadores.append(jugador)

            # Leer la tabla HTML de la URL
            df = pd.read_html(url, attrs={"id":"stats_standard_21"}, header=1)[0]

            # Agregar una nueva columna con el nombre del equipo
            df['Equipo'] = nombre_equipo

            # Crear un DataFrame a partir de la lista de jugadores
            df_jugadores = pd.DataFrame(jugadores)

            # Unir el DataFrame de las estadísticas estándar con el DataFrame de los jugadores
            df = pd.concat([df, df_jugadores], axis=1)

            return df
        else:
            print("Status Code %d" % statusCode)
            return None
    except:
        print("Error al hacer la solicitud, reintentando...")
        return None

In [41]:
# Para mejorar la extración la dividimos para hacerla por partes
df_p1 = extraccion_automatica(parte_1)

In [47]:
df_p2 = extraccion_automatica(parte_2)

In [56]:
df_p3 = extraccion_automatica(parte_3)

In [61]:
df_p4 = extraccion_automatica(parte_4)

In [66]:
df_p5 = extraccion_automatica(parte_5)

In [69]:
df_p6 = extraccion_automatica(parte_6)

Error al hacer la solicitud, reintentando...


In [70]:
# Copia del df original
df_copia1 = df_p1.copy()

df_copia2 = df_p2.copy()

df_copia3 = df_p3.copy()

df_copia4 = df_p4.copy()

df_copia5 = df_p5.copy()

df_copia6 = df_p6.copy()


In [71]:
def limpieza(df, nombre_archivo):
# Selección de la información básica
  df=df[['Jugador', 'País', 'Posc', 'Edad', 'Equipo', 'Enlace']]
# Nos quedamos con la nacionalidad en mayusculas
  df['País'] = df['País'].apply(lambda x: str(x)[-3:] if pd.notnull(x) else x)

# Drop de todos lo equipos
  df=df[(df['Jugador'] != 'Total del equipo') & (df['Jugador'] != 'Total del oponente')]

# Se queda con una posición por jugador
  df['Posc'] = df['Posc'].apply(lambda x: str(x)[-2:] if pd.notnull(x) else x)
# Limpiamos la edad
  df['Edad']=df['Edad'].apply(lambda x: str(x)[:2] if pd.notnull(x) else x)
# Limpiamos el nombre de los equipos
  df['Equipo'] = df['Equipo'].str.replace('Estadisticas-de-', '')
# Quitamos los - de la columna equipo
  df['Equipo'] = df['Equipo'].str.replace('-', ' ')

  # Añadir "https://fbref.com" al inicio de cada enlace
  df['Enlace'] = df['Enlace'].apply(lambda x: 'https://fbref.com' + str(x))



# Guardar el DataFrame en formato CSV
  df.to_csv(nombre_archivo + '.csv', index=False)
  print("DataFrame guardado en formato CSV con el nombre " + nombre_archivo + '.csv')

  df.to_excel(nombre_archivo + '.xlsx', index=False)
  print("DataFrame guardado en formato Excel con el nombre " + nombre_archivo + '.csv')


  return df

In [87]:
liga_argentina=limpieza(df_copia1, 'Liga_argentina_parte1')

DataFrame guardado en formato CSV con el nombre Liga_argentina_parte1.csv
DataFrame guardado en formato Excel con el nombre Liga_argentina_parte1.csv


In [88]:
liga_argentina_2=limpieza(df_copia2, 'Liga_argentina_parte2')

DataFrame guardado en formato CSV con el nombre Liga_argentina_parte2.csv
DataFrame guardado en formato Excel con el nombre Liga_argentina_parte2.csv


In [89]:
liga_argentina_3=limpieza(df_copia3, 'Liga_argentina_parte3')

DataFrame guardado en formato CSV con el nombre Liga_argentina_parte3.csv
DataFrame guardado en formato Excel con el nombre Liga_argentina_parte3.csv


In [90]:
liga_argentina_4=limpieza(df_copia4, 'Liga_argentina_parte4')

DataFrame guardado en formato CSV con el nombre Liga_argentina_parte4.csv
DataFrame guardado en formato Excel con el nombre Liga_argentina_parte4.csv


In [91]:
liga_argentina_5=limpieza(df_copia4, 'Liga_argentina_parte5')

DataFrame guardado en formato CSV con el nombre Liga_argentina_parte5.csv
DataFrame guardado en formato Excel con el nombre Liga_argentina_parte5.csv


In [92]:
liga_argentina_6=limpieza(df_copia4, 'Liga_argentina_parte6')


DataFrame guardado en formato CSV con el nombre Liga_argentina_parte6.csv
DataFrame guardado en formato Excel con el nombre Liga_argentina_parte6.csv


In [93]:
liga_argentina_d1 = pd.concat([liga_argentina, liga_argentina_2, liga_argentina_3, liga_argentina_4, liga_argentina_5, liga_argentina_6])

In [86]:
liga_argentina_d1

,Jugador,País,Posc,Edad,Equipo,Enlace
0,Claudio Corvalán,ARG,DF,35,Union,https://fbref.com/es/jugadores/8cd70adc/Claudi...
1,Franco Pardo,ARG,DF,27,Union,https://fbref.com/es/jugadores/de16bb25/-Franc...
2,Mauro Pittón,ARG,CC,29,Union,https://fbref.com/es/jugadores/363fa6bd/Mauro-...
3,Nicolás Orsini,ARG,DL,29,Union,https://fbref.com/es/jugadores/55934138/Nicola...
4,Nicolás Paz,ARG,DF,21,Union,https://fbref.com/es/jugadores/0af517c2/Nicola...
...,...,...,...,...,...,...
25,Matko Miljevic,USA,CC,23,Newells Old Boys,https://fbref.com/es/jugadores/74b76abc/Matko-...
26,Ramiro Macagno,ARG,PO,27,Newells Old Boys,https://fbref.com/es/jugadores/64293bed/Ramiro...
27,Lucas Mihovilcevich,ARG,DF,20,Newells Old Boys,https://fbref.com/es/jugadores/473928c3/Lucas-...
28,Jeremías Pérez Tica,ARG,DL,21,Newells Old Boys,https://fbref.com/es/jugadores/8260fd3e/Jeremi...
